In [1]:
from hashlib import sha256,blake2b
import os
from Crypto.Util.number import long_to_bytes,bytes_to_long

hash_bit_size = 288
hash_byte_size = 36

n = 288
k = 8
sample_num = 2**k
N = 2**(n//(k+1))

def xor(a,b):
    return bytes([i^^j for i,j in zip(a,b)])

def block_encrypt(block, times = 256):
    for i in range(times):
        sha256(block)
        
def bytes2bitvector(data, padlen = 256):
    bitstr = bin(int.from_bytes(data, "big"))[2:].zfill(padlen)
    return vector(GF(2), map(int,bitstr))

blake2b_hash = lambda data, hashlen = 36 : blake2b(data,digest_size = hashlen).digest()

# Linearization Attack for LGBP(n, k) with Random $k \ge n/2$ 

In [2]:
sample_num = int(hash_bit_size * 1.5)
msgs = [ os.urandom(32) for i in range(sample_num)]
cts = [ blake2b_hash(msg) for msg in msgs]
# cts = [ sha256(msg).digest() for msg in msgs]
vecs = [ bytes2bitvector(ct, padlen = hash_bit_size) for ct in cts]
M = matrix(GF(2), vecs)
M = M.T
M_kernel = M.right_kernel_matrix()

print(f"[+] {M_kernel.dimensions() = }")
best_norm = 999
for row in matrix(ZZ,M_kernel):
    if best_norm > row.norm(1):
        best_norm = row.norm(1)
        print(f"[+] {best_norm = }")
    idxs = row[:]
    msg_res = []
    hash_res = b"\x00"*hash_byte_size
    for i, idx in enumerate(idxs):
        if idx == 1:
            msg_res.append(msgs[i])
            hash_res = xor(cts[i], hash_res)
    assert bytes_to_long(hash_res) == 0, "bad row"
print(f"[+] {best_norm = }")

[+] M_kernel.dimensions() = (144, 432)
[+] best_norm = 149
[+] best_norm = 142
[+] best_norm = 133
[+] best_norm = 126
[+] best_norm = 126


In [3]:
# try native LLL to find shorter solutions but it does not work well on F2
# This is actually a LWC problem in https://decodingchallenge.org/
# For shorter solutions, try BJMM-ISD alg. or lattice alg. such as SBP(BKZ) 
L = matrix(ZZ, M_kernel)
IN = identity_matrix(ZZ, sample_num)*2
Mod_La = block_matrix(ZZ, [L, IN], nrows = 2)
print(f"Starting Reduction with {Mod_La.dimensions()}")
La = Mod_La.BKZ(block_size=40)
print("Lattice Reduction Done")

best_norm = 999
best_res = None
for row in La:
    norm = row.norm(1)
    if norm !=0 and (2 not in row) and (-2 not in row):
        if norm < best_norm:
            print(f"{norm = }")
            best_norm = norm
            best_res = row
print(f"{best_norm = }, { best_res = }")

idxs = (best_res)
msg_res = []
hash_res = b"\x00"*hash_byte_size
for i, idx in enumerate(idxs):
    if idx != 0:
        msg_res.append(msgs[i])
        hash_res = xor(cts[i], hash_res)
print(len(msg_res))
print(hash_res)  

Starting Reduction with (576, 432)
Lattice Reduction Done
norm = 143
norm = 130
norm = 128
norm = 121
best_norm = 121,  best_res = (1, 0, -1, -1, -1, 1, 0, -1, 0, 0, 0, 1, 1, -1, -1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, -1, 1, -1, 1, 0, 0, -1, 1, -1, 0, -1, 0, 0, 0, 0, 1, 0, 0, -1, 0, 1, 0, 0, -1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, -1, 0, 1, -1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, -1, -1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, -1, 0, 1, 0, -1, -1, 0, 1, 1, 0, 1, -1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, -1, -1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, -1, 0, 1, 0, -1, 0, 1, 0, -1, -1, 0, 1, 1, 0, 0, -1, 1, 0, 1, 0, 0, 1, 0, 0, -1, 1, -1, 1, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 1, 0, 0, -1, -1, 0, -1, 0, -1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, -1, -1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, -1, 0, 1, 1, -1, -1, -1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, 0, -1, 0, 0, -1, 0, 0, 0, 0, 0, 1, 0, 1, 0, -1, 0, 0, 0, 

# Linearization Attack for LGBP(n, k) with Fixed $k \ge n/2$ 

In [4]:
from Crypto.Util.number import long_to_bytes,bytes_to_long

def linearization_attack_with_fixed_k(n = 288, k = 2**8, sample_num = int(288*1.5)):
    assert n % 8 == 0 , "bad `n` parameter"
    print(f"[+] log fixed { k = }, { n = }, {sample_num = }")
    target_norm1 = (n//2) + 1
    hashbyte = n//8
    assert k > target_norm1, "bad `k` parameter"
    
    fixed_prefix = [os.urandom(32) for i in range(k - target_norm1)]
    prefix_hashs = [blake2b_hash(msg, hashbyte) for msg in fixed_prefix]
    fixed_cons = b"\x00"*(hashbyte)
    for h in prefix_hashs:
        fixed_cons = xor(fixed_cons,h)
    cons_vec = bytes2bitvector(fixed_cons)
    
    msgs = [ os.urandom(32) for i in range(sample_num)]
    cts = [blake2b_hash(msg, hashbyte) for msg in msgs]
    vecs = [ bytes2bitvector(xor(ct,fixed_cons), padlen = hash_bit_size) for ct in cts]
    M = matrix(GF(2), vecs)
    M = M.T
    M_kernel = M.right_kernel_matrix()
    L = matrix(ZZ,M_kernel)
    results = []
    for row in L:
        if row.norm(1) == target_norm1:
            results.append(row)
    print(f"[+] find { len(results) } valid solutions.")
    print("[+] Validation...")
    final_orignal_result = []
    for idxs in results:
        msg_res = fixed_prefix[:]
        hash_res = fixed_cons[:]
        for i, idx in enumerate(idxs):
            if idx != 0:
                msg_res.append(msgs[i])
                hash_res = xor(cts[i], hash_res)
        final_orignal_result.append(msg_res)
        assert bytes_to_long(hash_res) == 0, "error"
    for msgs in final_orignal_result:
        hash_res = b"\x00"*hashbyte
        for msg in msgs:
            hash_res = xor(hash_res, blake2b_hash(msg, hashbyte))
        assert bytes_to_long(hash_res) == 0 and len(msgs) == k, "not valid solutions"
    print(f"[+] find {len(final_orignal_result)} solutions for LGBP{n, len(msgs)}")

linearization_attack_with_fixed_k(n = 288, k = 150, sample_num = 288*5)

[+] log fixed  k = 150,  n = 288, sample_num = 1440
[+] find 47 valid solutions.
[+] Validation...
[+] find 47 solutions for LGBP(288, 150)
